In [32]:
import setup_django
setup_django.init()

In [33]:
import pandas as pd
from django.conf import settings

ratings_path = settings.DATA_DIR / "ratings_small.csv" #pathlib
ratings_path.exists()

True

In [34]:
df = pd.read_csv(ratings_path)

In [35]:
df.head(n=10)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [36]:
from django.contrib.auth import get_user_model

User = get_user_model()

In [37]:
current_users = User.objects.all().values_list('id', flat=True)
rating_users = df['userId'].tolist()

In [38]:
missing_user_ids = set(rating_users) - set(current_users)
missing_user_ids

set()

In [39]:
for uid in missing_user_ids:
    User.objects.create(
        id=uid,
        username=f"missing-user-{uid}"
    )

In [40]:
import math
from decimal import Decimal

In [41]:
df['value'] = df['rating'].apply(lambda x: math.ceil(Decimal(x)))
df.head()

,userId,movieId,rating,timestamp,value
0,1,31,2.5,1260759144,3
1,1,1029,3.0,1260759179,3
2,1,1061,3.0,1260759182,3
3,1,1129,2.0,1260759185,2
4,1,1172,4.0,1260759205,4


In [42]:
df['user_id'] = df['userId']
df['object_id'] = df['movieId']

In [43]:
df.head()

,userId,movieId,rating,timestamp,value,user_id,object_id
0,1,31,2.5,1260759144,3,1,31
1,1,1029,3.0,1260759179,3,1,1029
2,1,1061,3.0,1260759182,3,1,1061
3,1,1129,2.0,1260759185,2,1,1129
4,1,1172,4.0,1260759205,4,1,1172


In [44]:
cols = ['user_id', 'value', 'object_id']
transformed_df = df.copy()[cols]
rating_records = transformed_df.to_dict('records')

In [45]:
from ratings.models import Rating
qs = Rating.objects.all()
qs.delete()

(0, {})

In [46]:
from django.contrib.contenttypes.models import ContentType

ctype = ContentType.objects.get(app_label='movies', model='movie')

In [47]:
new_ratings = []
for r in rating_records:
    r['content_type'] = ctype
    new_ratings.append(
        Rating(**r)
    )
Rating.objects.bulk_create(new_ratings, ignore_conflicts=True, batch_size=1000)

[<Rating: admin | 3>,
 <Rating: admin | 3>,
 <Rating: admin | 3>,
 <Rating: admin | 2>,
 <Rating: admin | 4>,
 <Rating: admin | 2>,
 <Rating: admin | 2>,
 <Rating: admin | 2>,
 <Rating: admin | 4>,
 <Rating: admin | 2>,
 <Rating: admin | 3>,
 <Rating: admin | 1>,
 <Rating: admin | 4>,
 <Rating: admin | 4>,
 <Rating: admin | 3>,
 <Rating: admin | 2>,
 <Rating: admin | 2>,
 <Rating: admin | 3>,
 <Rating: admin | 1>,
 <Rating: admin | 3>,
 <Rating: erik41 | 4>,
 <Rating: erik41 | 5>,
 <Rating: erik41 | 5>,
 <Rating: erik41 | 4>,
 <Rating: erik41 | 4>,
 <Rating: erik41 | 3>,
 <Rating: erik41 | 3>,
 <Rating: erik41 | 4>,
 <Rating: erik41 | 3>,
 <Rating: erik41 | 5>,
 <Rating: erik41 | 4>,
 <Rating: erik41 | 3>,
 <Rating: erik41 | 3>,
 <Rating: erik41 | 3>,
 <Rating: erik41 | 3>,
 <Rating: erik41 | 3>,
 <Rating: erik41 | 3>,
 <Rating: erik41 | 5>,
 <Rating: erik41 | 1>,
 <Rating: erik41 | 3>,
 <Rating: erik41 | 3>,
 <Rating: erik41 | 3>,
 <Rating: erik41 | 4>,
 <Rating: erik41 | 4>,
 <Rating